In [12]:
pip install speechbrain


    extract-msg (<=0.29.*)
                 ~~~~~~~^
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.8 MB/s eta 0:00:00
    extract-msg (<=0.29.*)
                 ~~~~~~~^
Note: you may need to restart the kernel to use updated packages.


In [26]:
import os
import librosa
import soundfile as sf
import torch
import numpy as np
from speechbrain.inference import SpeakerRecognition
from sklearn.metrics.pairwise import cosine_similarity


In [22]:
DATA_DIR = "data/train/"
TARGET_SR = 16000


In [24]:
def preprocess_audio(path):
    audio, sr = librosa.load(path, sr=None)
    if sr != TARGET_SR:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=TARGET_SR)
    audio, _ = librosa.effects.trim(audio)  # Trim silence
    audio = audio / max(abs(audio))  # Normalize
    return audio
def process_all_audio():
    for root, _, files in os.walk(DATA_DIR):
        for file in files:
            if file.endswith(".wav"):
                full_path = os.path.join(root, file)
                print(f"Processing: {full_path}")
                audio = preprocess_audio(full_path)
                sf.write(full_path, audio, TARGET_SR)

if __name__ == "__main__":
    process_all_audio()

Processing: data/train/s4.wav
Processing: data/train/s5.wav
Processing: data/train/s7.wav
Processing: data/train/s6.wav
Processing: data/train/s2.wav
Processing: data/train/s3.wav
Processing: data/train/s1.wav
Processing: data/train/s8.wav


In [30]:

model = SpeakerRecognition.from_hparams(
    source="speechbrain/spkrec-ecapa-voxceleb",
    savedir="tmp/spkrec-ecapa"
)

def get_embedding(file_path):
    signal = model.load_audio(file_path)
    embedding = model.encode_batch(signal)
    return embedding.squeeze().detach().cpu().numpy()

# Step 1: Register known speakers
speaker_db = {}
DATA_DIR = "data/train/"

for speaker in os.listdir(DATA_DIR):
    speaker_path = os.path.join(DATA_DIR, speaker)
    if os.path.isdir(speaker_path):
        embeddings = []
        for file in os.listdir(speaker_path):
            if file.endswith(".wav"):
                emb = get_embedding(os.path.join(speaker_path, file))
                embeddings.append(emb)
        speaker_db[speaker] = np.mean(embeddings, axis=0)  # Average embedding
# Step 2: Identify a speaker from test audio
def identify(file_path):
    test_emb = get_embedding(file_path).reshape(1, -1)
    best_score = -1
    best_match = None
    for speaker, emb in speaker_db.items():
        score = cosine_similarity(test_emb, emb.reshape(1, -1))[0][0]
        if score > best_score:
            best_score = score
            best_match = speaker
    return best_match, best_score

# Test it
test_file = "data/test/s4.wav"  # Replace with your test file
speaker, score = identify(test_file)
print(f"Identified Speaker: {speaker} (Score: {score:.4f})")

Identified Speaker: None (Score: -1.0000)
